# PROJEKTS: Knowledge Hub



Situācija
Tu esi tikko pieņemts darbā par MI Inženieri. Uzņēmuma vadītājs tev iedevis mapi UNSORTED_DATA, kurā ir haoss – sapulču ieraksti, tāfeles fotogrāfijas un līgumu melnraksti.

Nevienam nav laika to visu lasīt vai klausīties.

## Tavs Uzdevums:

Uzbūvēt Multimodālu RAG Sistēmu, kas "apēd" šos failus un ļauj direktoram čatā uzdot jautājumus (piemēram: "Cik mēs esam parādā?" vai "Ko nolēma par mārketingu?"), saņemot precīzas atbildes ar atsaucēm.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:


from pathlib import Path
BASE = Path("/content/drive/MyDrive/Colab Notebooks/BDA_M1/Lab26_KnowledgeHub")
#df = pd.read_csv(BASE / "data" / "raw" / "mani_dati.csv")

# Pārbauda, vai norādītā mape eksistē
print("Mape eksistē:", BASE.exists())
print("Bāzes ceļš:", BASE)

if BASE.exists():
    # Parāda mapes saturu šajā līmenī
    print("\nSaturs mapē:")
    for p in sorted(BASE.iterdir()):
        print("MAPE:" if p.is_dir() else "FAILS:", p.name)

    # Parāda visus failus arī apakšmapēs
    #print("\nVisi faili apakšmapēs:")
    #for f in sorted(BASE.rglob("*")):
    #    if f.is_file():
    #        print(f.relative_to(BASE))
else:
    print("\nNorādītais ceļš nav atrasts. Pārbaudi ceļu pēc MyDrive.")


Mape eksistē: True
Bāzes ceļš: /content/drive/MyDrive/Colab Notebooks/BDA_M1/Lab26_KnowledgeHub

Saturs mapē:
MAPE: .git
FAILS: Lab26_4.modulis_JurijsHolms_KnowledgeHub.ipynb
FAILS: meeting.mp3
FAILS: project.pdf
FAILS: whiteboard.png
